# Coursera Capstone Project

## The Battle of the Neighborhoods - Week 1

Let's import the libraries we will need first

In [2]:
import numpy as np
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
import requests 
from pandas.io.json import json_normalize 

from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

# Introduction and Data Section

## 1. Introduction

### 1.1 Scenario

I have friend who lives in Singapore, she has been offered by her employer to relocate to Manhattan NY. She is very excited about it but she is also hesitant because she loves her current residence in Singapore and doubts she will find a similarly convinient place in NYC.


She lives within walking distance to an MRT metro station, different kinds of restaurants, cafes, shop and many kinds of entertainment. 


I am excited for this opportunity to help my friend find a place in NY similar to her place in Singapore now while also applying what I learned in the Coursera Applied Data Science Capstone course.

### 1.2 Business Problem

**To find an apartment in Manhattan NY that meets the following:**
- 2 or 3 bedrooms
- a metro station within a 1.0 mile radius
- rent: USD7,000 per month
- accesible places: coffee shop, asian restaurants, groceries, gym

### 1.3 Interested Audience

Aside from my friend and me, this study may alse be useful to anyone interested in moving to Manhattan NY. It is also a good reference for anyone wanting to practice and develop their Data Science skills

## 2. Data Section

### 2.1 Data Description

**We need the following data to answer our Business Problem**

- List of Boroughs and neighborhoods of Manhattan with their geodata (latitude and longitude)
- Venues for each Manhattan neighborhood
- List of Subway metro stations in Manhattan with their address location
- List of apartments for rent in Manhattan area with their addresses, price, and details

### 2.2 Data Usage - How the data will be used to solve the problem

**The data will be used as follows:**

- Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups
- Use foursquare and geopy data to map the location of subway metro stations, explore each subway location to identify venues near each one
- Use Foursquare and geopy data to map the location of rental places
- Make a map that shows average rental price per square ft within a radious of 1.0 mile around each subway station
- Use Nominatim and Geopy-distance to convert addresses from rental locations to geodata
- We will also look for availble open data sources from real estate sites or goverment data

**After we process the data we will be able to asnwer the following questions to help us solve our Business Problem:**

- How much is the cost of rent (per square ft) around a mile radius from each subway metro station?
- Which area of Manhattan has the best rental prices and meets our requirements?
- How venues distribute among Manhattan neighborhoods and around metro stations?
- Any other interesting statistical data findings of the real estate and overall data.

**Reference of venues around current residence in Singapore for comparison to Manhattan place**

In [4]:
address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore place are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Singapore place are 1.2795465, 103.8476049.


In [5]:
neighborhood_latitude=1.2792655
neighborhood_longitude=103.8480938

In [16]:
# The code was removed by Watson Studio for sharing.

In [10]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=VBBXDVFCTI4QAQ5USY3ID3YEEBEYZKQ0SP4TL3MRZ0V0N1O2&client_secret=V4JSSKC0WZWHNZFLDL4PI5ZFL5VWAQY0FXWW1JKSR1IO2LHZ&v=20200722&ll=1.2792655,103.8480938&radius=500&limit=100'

In [11]:
results = requests.get(url).json()

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Napoleon Food & Wine Bar,Wine Bar,1.279925,103.847333
1,Native,Cocktail Bar,1.280135,103.846844
2,Park Bench Deli,Deli / Bodega,1.279872,103.847287
3,Freehouse,Beer Garden,1.281254,103.848513
4,Pepper Bowl,Asian Restaurant,1.279371,103.846710
5,Meat Smith,Southern / Soul Food Restaurant,1.280205,103.847410
6,Anglo Indian Cafe & Bar,Indian Restaurant,1.279084,103.850127
7,Mellower Coffee,Café,1.277814,103.848188
8,PS.Cafe,Café,1.280468,103.846264
9,Lau Pa Sat Satay Street,Street Food Gathering,1.280261,103.850235


**Map of Singapore with venues near residence place**


In [14]:
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg